# Content-based

In [1]:
#import vk                                                                       
import time                                                                     
import pickle                                                                   
import numpy as np 
import collections
import itertools
from sklearn.metrics.pairwise import cosine_similarity

from scipy.sparse import coo_matrix, csr_matrix, csc_matrix
from collections import defaultdict                                             
                                                                                
def save_obj(obj, name ):                                                       
    with open(name + '.pkl', 'wb') as f:                                        
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)                            
                                                                                
def load_obj(name ):                                                            
    with open(name + '.pkl', 'rb') as f:                                        
        return pickle.load(f)

In [2]:
all_members = {}
all_members.update(load_obj('./Anya_persons'))
print(len(all_members))
all_members.update(load_obj('./Dasha_persons'))
print(len(all_members))
all_members.update(load_obj('./Maks_persons'))

15209
31037


In [3]:
users_with_no_inf = []
groups_l = set()
for pers in all_members:
    try:
        groups_l.update(all_members[pers]['groups']) 
    except Exception: users_with_no_inf.append(pers)
groups_l = list(groups_l)

In [4]:
groups = {idx:group for idx, group in enumerate(groups_l)}
reverse_groups = {group:idx for idx, group in enumerate(groups_l)}

In [5]:
print(max(groups.keys()))
print(max(reverse_groups.keys()))

1141946
189802470


In [5]:
for pers in users_with_no_inf:
    del all_members[pers]

In [6]:
users_l = list(all_members.keys())
users = {idx:user for idx, user in enumerate(users_l)}
reverse_users = {user:idx for idx, user in enumerate(users_l)}

In [9]:
print(len(all_members))

46181


In [7]:
data = []
row = []
col =[]
for idx, (user, val) in enumerate(all_members.items()):
    for group in val['groups'][0:int(0.03*len(val['groups']))]:
        row.append(idx)
        data.append(100)
        col.append(reverse_groups[group])
    for group in val['groups'][int(0.03*len(val['groups'])):int(0.1*len(val['groups']))]:
        row.append(idx)
        data.append(10)
        col.append(reverse_groups[group])
    for group in val['groups'][int(0.1*len(val['groups'])):]:
        row.append(idx)
        data.append(1)
        col.append(reverse_groups[group])

In [8]:
csr = csr_matrix((np.array(data), (np.array(row),np.array(col))))

In [9]:
csr.shape

(46181, 1141947)

In [10]:
from scipy import spatial
from lightfm import LightFM
from sklearn.metrics import pairwise_distances
from scipy.sparse import coo_matrix, csr_matrix, load_npz
from sklearn.metrics.pairwise import cosine_similarity
model = LightFM(loss='warp', no_components=100, learning_rate=0.03, learning_schedule="adadelta")

In [ ]:
model.fit(coo_matrix(csr), epochs=100, num_threads=40, verbose=True)

In [12]:
user_feature_bias, user_feature_embeddings = model.get_user_representations()
item_feature_bias, item_feature_embeddings = model.get_item_representations()

In [13]:
def precision_for_one(recomend, user_l):
    recomend = set(recomend)
    groups_of_user = set(all_members[user_l]['groups'])
    TP = len(recomend&groups_of_user)
    return TP/6.

In [44]:
import random
users_for_test = random.sample(users_l, 100)

In [56]:
def content_based_recomendation(user_l, item_feature_embeddings):
    groups_for_recommend = []
    groups_of_user = all_members[user_l]['groups']
    groups_of_user_revers = []
    for i in groups_of_user[0:6]:
         groups_of_user_revers.append(reverse_groups[i])
    for i in groups_of_user_revers:
        metrics = list(cosine_similarity(item_feature_embeddings[i].reshape(1, -1),  item_feature_embeddings)[0])   
        sorted_keys = sorted(range(len(metrics)), key=metrics.__getitem__, reverse = True)
        for j, val in enumerate(sorted_keys):
            if val != i and (groups[val] not in groups_for_recommend) and (groups[val] not in groups_of_user): 
                best_group = val
                break
        groups_for_recommend.append(groups[best_group])
    return groups_for_recommend


In [29]:
content_based_recomendation(9183085, item_feature_embeddings)

[161180646, 23611958, 127430746, 18517784, 354372, 132528325]

In [ ]:
#делаем рекомендации для группы из 100 человек и считаем средний precision
list_of_prec = []
for user_l in users_for_test:
    recomend = content_based_recomendation(user_l, item_feature_embeddings)    
    list_of_prec.append(precision_for_one(recomend, user_l))

In [48]:
sum(list_of_prec)/len(list_of_prec)

0.1966666666666668

In [58]:
content_based_recomendation(9183085, item_feature_embeddings)   

[161180646, 69547083, 155161349, 18517784, 48149358, 144437844]

In [31]:
friends_for_test = []

In [32]:
with open('All_friends.txt' , 'r') as f:
    for line in f:
        friends_for_test.append(int(line))   

In [36]:
dict_friends_test = dict()
for pers in friends_for_test:
    if pers in users_l:
        dict_friends_test[pers] = dict()
        dict_friends_test[pers]['groups'] = content_based_recomendation(pers, item_feature_embeddings)   
        dict_friends_test[pers]['method'] = 3

In [38]:
with open('Dasha_recs_3.pkl', 'wb') as f:
    pickle.dump(dict_friends_test , f)